## Write a file with PyNWB

In [1]:
from pynwb import NWBFile
from datetime import datetime
from dateutil import tz

session_start_time = datetime.now()

nwbfile = NWBFile(
    session_description='Description',
    identifier='Mouse',
    session_start_time=session_start_time,
)
print(nwbfile)

root pynwb.file.NWBFile at 0x4378067408
Fields:
  file_create_date: [datetime.datetime(2021, 3, 29, 14, 33, 28, 96016, tzinfo=tzlocal())]
  identifier: Mouse
  session_description: Description
  session_start_time: 2021-03-29 14:33:28.094479+02:00
  timestamps_reference_time: 2021-03-29 14:33:28.094479+02:00



/Users/legouee/miniconda3/lib/python3.7/site-packages/pynwb/file.py:753: UserWarning: Date is missing timezone information. Updating to local timezone.
  warn("Date is missing timezone information. Updating to local timezone.")


In [2]:
from pynwb.device import Device
from pynwb.ophys import OpticalChannel

device = nwbfile.create_device(
    name='Microscope', 
    description='My two-photon microscope',
    manufacturer='The best microscope manufacturer'
)
optical_channel = OpticalChannel(
    name='OpticalChannel', 
    description='an optical channel', 
    emission_lambda=500.
)
imaging_plane = nwbfile.create_imaging_plane(
    name='ImagingPlane',
    optical_channel=optical_channel,
    imaging_rate=30.,
    description='a very interesting part of the brain',
    device=device,
    excitation_lambda=600.,
    indicator='GFP',
    location='V1',
    grid_spacing=[.01, .01],
    grid_spacing_unit='meters',
    origin_coords=[1., 2., 3.],
    origin_coords_unit='meters'
)

In [3]:
from pynwb.ophys import TwoPhotonSeries
import numpy as np

image_series = TwoPhotonSeries(
    name='TwoPhotonSeries1',
    data=np.ones((2,3,4)),
    imaging_plane=imaging_plane,
    rate=1.0,
    unit='normalized amplitude'
)

print(image_series)

TwoPhotonSeries1 pynwb.ophys.TwoPhotonSeries at 0x4590349136
Fields:
  comments: no comments
  conversion: 1.0
  data: [[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]
  description: no description
  imaging_plane: ImagingPlane pynwb.ophys.ImagingPlane at 0x4590350288
Fields:
  conversion: 1.0
  description: a very interesting part of the brain
  device: Microscope pynwb.device.Device at 0x4590349712
Fields:
  description: My two-photon microscope
  manufacturer: The best microscope manufacturer

  excitation_lambda: 600.0
  imaging_rate: 30.0
  indicator: GFP
  location: V1
  optical_channel: (
    OpticalChannel <class 'pynwb.ophys.OpticalChannel'>
  )
  unit: meters

  rate: 1.0
  resolution: -1.0
  starting_time: 0.0
  starting_time_unit: seconds
  unit: normalized amplitude



In [4]:
nwbfile.add_acquisition(image_series)

In [5]:
from pynwb import NWBHDF5IO

with NWBHDF5IO('Example_NWB_File.nwb', 'w') as io:
    io.write(nwbfile)

## Read this file with Neo NWBIO

In [6]:
import neo
from neo import *
import pynwb
from pynwb import *
import quantities as pq
from quantities import s, ms, kHz, Hz, uV
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from os.path import exists
from urllib.request import urlretrieve

print("neo = ", neo.__version__)
print("pynwb = ", pynwb.__version__)

filename= "Example_NWB_File.nwb"
ior = NWBIO(filename=filename, mode='r')
retrieved_blocks = ior.read_all_blocks()
print("retrieved_blocks = ", retrieved_blocks)

neo =  0.9.0
pynwb =  1.4.0
retrieved_blocks =  [<neo.core.block.Block object at 0x11a9fa410>]


In [7]:
print(retrieved_blocks[0].segments[0].imagesequences[0].__dict__)

{'_dimensionality': Dimensionality({meter: 1}), 'annotations': {'nwb_neurodata_type': ('pynwb.ophys', 'TwoPhotonSeries'), 'imaging_plane': {'indicator': 'GFP', 'name': 'ImagingPlane', 'excitation_lambda': 600.0, 'optical_channel': {'name': 'OpticalChannel', 'emission_lambda': 500.0, 'description': 'an optical channel'}, 'device': {'name': 'Microscope', 'manufacturer': 'The best microscope manufacturer', 'description': 'My two-photon microscope'}, 'location': 'V1', 'description': 'a very interesting part of the brain', 'reference_frame': None, 'imaging_rate': 30.0}, 'rate': 1.0, 'unit': 'normalized amplitude', 'starting_time': 0.0}, 'array_annotations': {}, 'name': 'TwoPhotonSeries1', 'file_origin': None, 'description': None, 'segment': <neo.core.segment.Segment object at 0x11a9f68d0>, 'channel_index': None, 'sampling_rate': array(30.) * Hz, 'spatial_scale': 'No spatial_scale', '_t_start': array(0.) * s}


In [8]:
print(retrieved_blocks[0].segments[0].imagesequences[0].annotations['imaging_plane']['device']['name'])

Microscope


## Write a new NWB file with Neo NWBIO

In [9]:
filename2="Example_Write_same_NWB_file.nwb"
iow = NWBIO(filename2, mode='w')
iow

In [10]:
iow2 = iow.write_all_blocks(retrieved_blocks)

## Read this file with PyNWB

In [11]:
io_pynwb = pynwb.NWBHDF5IO(filename2, mode='r')
print("io_pynwb = ", io_pynwb)
print("pynwb.__version__ = ", pynwb.__version__)

io_pynwb =  <pynwb.NWBHDF5IO object at 0x11b04b450>
pynwb.__version__ =  1.4.0


In [12]:
io_pynwb_r = io_pynwb.read()

In [13]:
io_pynwb.read_builder()

root GroupBuilder {'attributes': {'namespace': 'core', 'neurodata_type': 'NWBFile', 'nwb_version': '2.2.5', 'object_id': 'b902d34f-1a86-4a66-946d-d4789f322df9'}, 'groups': {'acquisition': root/acquisition GroupBuilder {'attributes': {}, 'groups': {'ImagingPlane': root/acquisition/ImagingPlane GroupBuilder {'attributes': {'comments': 'no comments', 'description': 'no description', 'namespace': 'core', 'neurodata_type': 'TwoPhotonSeries', 'object_id': 'fafa55ca-a220-43cc-ba8f-9d9c1610e8be'}, 'groups': {}, 'datasets': {'data': root/acquisition/ImagingPlane/data DatasetBuilder {'attributes': {'conversion': 1.0, 'resolution': -1.0, 'unit': 'normalized amplitude'}, 'data': <HDF5 dataset "data": shape (2, 4, 3), type "<i8">}, 'starting_time': root/acquisition/ImagingPlane/starting_time DatasetBuilder {'attributes': {'rate': 1.0, 'unit': 'seconds'}, 'data': 0.0}}, 'links': {'imaging_plane': root/acquisition/ImagingPlane/imaging_plane LinkBuilder {'builder': root/general/optophysiology/ImagingP

In [14]:
io_pynwb_r.acquisition

{'ImagingPlane': ImagingPlane pynwb.ophys.TwoPhotonSeries at 0x4750269776
 Fields:
   comments: no comments
   conversion: 1.0
   data: <HDF5 dataset "data": shape (2, 4, 3), type "<i8">
   description: no description
   imaging_plane: ImagingPlane pynwb.ophys.ImagingPlane at 0x4750268496
 Fields:
   conversion: 1.0
   description: a very interesting part of the brain
   device: Microscope pynwb.device.Device at 0x4750269136
 Fields:
   description: My two-photon microscope
   manufacturer: The best microscope manufacturer
 
   excitation_lambda: 600.0
   imaging_rate: 30.0
   indicator: GFP
   location: V1
   optical_channel: (
     OpticalChannel <class 'pynwb.ophys.OpticalChannel'>
   )
   unit: meters
 
   rate: 1.0
   resolution: -1.0
   starting_time: 0.0
   starting_time_unit: seconds
   unit: normalized amplitude}